In [78]:
# Data Clean
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data = train_data.drop(columns = ['zipcode'])
test_data = test_data.drop( columns = ['id','date','zipcode'])
# ID date zipcode
test_data.columns

Index(['Unnamed: 0', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [79]:
#C2.1
X = train_data.drop(columns = ['Unnamed: 0','price'])
y = train_data['price']

X_const_train = sm.add_constant(X)
model = sm.OLS(y, X_const_train).fit()
print("Coefficients:\n", model.params)

y_pred_train = model.predict(X_const_train)

mse = mean_squared_error(y, y_pred_train)
r2 = model.rsquared


print("Training MSE:", mse)
print("Training R^2:", r2)

print(model.summary())


Coefficients:
 const           -2.308890e+07
bedrooms        -1.470428e+04
bathrooms        2.568778e+04
sqft_living      8.308361e+01
sqft_lot         3.759298e-01
floors           1.555558e+04
waterfront       7.155352e+05
view             6.302790e+04
condition        1.881640e+04
grade            7.953460e+04
sqft_above       4.201109e+01
sqft_basement    4.107431e+01
yr_built        -2.400669e+03
yr_renovated     4.368294e+01
lat              5.535050e+05
long            -7.424027e+03
sqft_living15    6.801579e+01
sqft_lot15      -5.155276e-01
dtype: float64
Training MSE: 31486167775.794846
Training R^2: 0.7265334318706022
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     163.2
Date:                Sun, 15 Feb 2026   Prob (F-

C2.1 The MSE is 31486167775.794846 on training data, and the r squared value is 0.7265334318706022


In [80]:
# C 2.2
from sklearn.metrics import r2_score

X = test_data.drop(columns = ['Unnamed: 0','price'])
y = test_data['price']
X_const_test = sm.add_constant(X)

y_pred_test = model.predict(X_const_test)

mse = mean_squared_error(y, y_pred_test)
r2 = r2_score(y, y_pred_test)

print("Testing MSE:", mse)
print("Testing R^2:", r2)

print(model.summary())


Testing MSE: 57628154705.66643
Testing R^2: 0.6543560876121193
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     163.2
Date:                Sun, 15 Feb 2026   Prob (F-statistic):          2.71e-263
Time:                        22:33:51   Log-Likelihood:                -13505.
No. Observations:                1000   AIC:                         2.704e+04
Df Residuals:                     983   BIC:                         2.713e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

C 2.2
The MSE is 57628154705.66643 and the R squared value is 0.6543560876121193

C 2.3

The r squared value didnt see to much change between train and test, showing that the model was decently fit. Test MSE/Train MSE was around 1.83 which is reason too believe the model is somewhat overfit. The features that contribute the most are the ones with lowest p-values, such as sqft_basement, sqft_above, waterfront, and view, all with p-values at or rounded too 0.00. The test MSE is 1.8 the train MSE showing that the model might be overfit on trian data, and needs further test data. The goal is for this number to be as close to one as possible.



In [81]:
#C 3
import numpy as np

X_train = train_data.drop(columns = ['Unnamed: 0','price']).to_numpy()
y_train = train_data['price'].to_numpy()

X_test = test_data.drop(columns = ['Unnamed: 0','price']).to_numpy()
y_test = test_data['price'].to_numpy()

X_train = np.c_[np.ones(len(X_train)), X_train]
X_test  = np.c_[np.ones(len(X_test)),  X_test]

theta = np.linalg.pinv(X_train) @ y_train

yhat_train = X_train @ theta
yhat_test  = X_test @ theta

test_MSE = mean_squared_error(y_test, yhat_test)
test_R2 = r2_score(y_test, yhat_test)
train_MSE = mean_squared_error(y_train, yhat_train)
train_R2 = r2_score(y_train, yhat_train)


print("Training MSE:", train_MSE)
print("Training R^2:", train_R2)
print("Testing MSE:", test_MSE)
print("Testing R^2:", test_R2)





Training MSE: 31486167775.794846
Training R^2: 0.726533431870602
Testing MSE: 57628154705.66643
Testing R^2: 0.6543560876121193


C3, the models MSE and R2 Match for both testing and Training, demonstrating the behind the scenes calculations of linear regression models.

In [82]:
#C4
def make_poly_features(x, p):

    x = np.asarray(x).reshape(-1, 1)

    X_poly = np.ones((x.shape[0], p + 1))

    for k in range(1, p + 1):
        X_poly[:, k] = x[:, 0] ** k

    return X_poly

def fit_closed_form(X, y):
    y = np.asarray(y).reshape(-1, 1)
    return np.linalg.pinv(X) @ y

def predict(X, theta):
    return X @ theta

x_train = train_data["sqft_living"].to_numpy()
y_train = train_data["price"].to_numpy()

x_test  = test_data["sqft_living"].to_numpy()
y_test  = test_data["price"].to_numpy()

degrees = [1, 2, 3, 5]
rows = []

for p in degrees:
    Xtr = make_poly_features(x_train, p)
    Xte = make_poly_features(x_test, p)

    theta = fit_closed_form(Xtr, y_train)

    yhat_tr = predict(Xtr, theta)
    yhat_te = predict(Xte, theta)

    mse_tr = mean_squared_error(y_train, yhat_tr)
    r2_tr  = r2_score(y_train, yhat_tr)

    mse_te = mean_squared_error(y_test, yhat_te)
    r2_te  = r2_score(y_test, yhat_te)

    rows.append([p, mse_tr, r2_tr, mse_te, r2_te])

results = pd.DataFrame(
    rows,
    columns=["Degree p", "MSE Train", "R² Train", "MSE Test", "R² Test"]
)

results

,Degree p,MSE Train,R² Train,MSE Test,R² Test
0,1,5.794753e+10,0.496709,8.857598e+10,0.468736
1,2,5.482267e+10,0.523849,7.179168e+10,0.569406
2,3,5.378519e+10,0.532860,9.983348e+10,0.401216
3,5,5.411491e+10,0.529996,9.326506e+13,-558.388064


C4: As the degree P increases from 1 to 2 the test r2 increases and the mse decrease showcaseing a more accurate model. When degree goes past 3 the returns diminish, r2 drops around from 0.569 to 0.401(when degree of p is 3). When the degree level is 5, the r2 value falls below zero signaling the model prediction error is larger than the baseline error.

In [83]:
# C 5.1
def add_intercept(X):
    X = np.asarray(X)
    return np.c_[np.ones((X.shape[0], 1)), X]

def standardize_train_test(X_train_raw, X_test_raw):

    X_train_raw = np.asarray(X_train_raw, dtype=float)
    X_test_raw  = np.asarray(X_test_raw, dtype=float)

    mu = X_train_raw.mean(axis=0)
    sigma = X_train_raw.std(axis=0)
    sigma[sigma == 0] = 1.0

    X_train = (X_train_raw - mu) / sigma
    X_test  = (X_test_raw - mu) / sigma
    return X_train, X_test, mu, sigma

def gradient_descent(X, y, alpha, iters):

    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1, 1)
    N, d = X.shape

    theta = np.zeros((d, 1), dtype=float)

    for _ in range(iters):
        grad = (2.0 / N) * (X.T @ (X @ theta - y))
        theta = theta - alpha * grad


        if not np.isfinite(theta).all():
            break

    return theta

# C 5.2
def eval_model(X, y, theta):
    y = np.asarray(y, dtype=float).reshape(-1, 1)
    yhat = np.asarray(X, dtype=float) @ np.asarray(theta, dtype=float)


    if not np.isfinite(yhat).all():
        return np.inf, -np.inf

    mse = mean_squared_error(y, yhat)
    r2 = r2_score(y, yhat)
    return mse, r2


feature_cols = [c for c in train_data.columns if c not in ["price", "Unnamed: 0"]]

X_train_raw = train_data[feature_cols].to_numpy()
y_train = train_data["price"].to_numpy().reshape(-1, 1)

X_test_raw = test_data[feature_cols].to_numpy()
y_test = test_data["price"].to_numpy().reshape(-1, 1)


X_train_scaled, X_test_scaled, mu, sigma = standardize_train_test(X_train_raw, X_test_raw)
X_train = add_intercept(X_train_scaled)
X_test  = add_intercept(X_test_scaled)




alphas = [0.01, 0.1, 0.5]
iters_list = [10, 50, 100]

rows = []
for alpha in alphas:
    for iters in iters_list:
        theta = gradient_descent(X_train, y_train, alpha, iters)

        mse_tr, r2_tr = eval_model(X_train, y_train, theta)
        mse_te, r2_te = eval_model(X_test, y_test, theta)


        rows.append([
            alpha,
            iters,
            theta.flatten()[:5],
            mse_tr, r2_tr,
            mse_te, r2_te
        ])

results = pd.DataFrame(
    rows,
    columns=["alpha", "iters", "theta(first 5)", "MSE Train", "R2 Train", "MSE Test", "R2 Test"]
)

results


,alpha,iters,theta(first 5),MSE Train,R2 Train,MSE Test,R2 Test
0,0.01,10,"[95198.02483770325, 11928.618743287172, 20283....",2.357278e+11,-1.047365e+00,2.805687e+11,-6.828036e-01
1,0.01,50,"[330895.53038963006, 6005.279760915576, 23821....",6.972050e+10,3.944571e-01,9.704954e+10,4.179133e-01
2,0.01,100,"[451397.6498338787, -3656.919333420636, 19241....",3.682035e+10,6.802045e-01,6.333304e+10,6.201392e-01
3,0.10,10,"[464535.71669041866, -4645.6314527042505, 1873...",3.510510e+10,6.951019e-01,6.163043e+10,6.303511e-01
4,0.10,50,"[520407.4063912902, -12457.193432165415, 17621...",3.149726e+10,7.264371e-01,5.772248e+10,6.537904e-01
5,0.10,100,"[520414.83389399067, -12519.07659846653, 18424...",3.148643e+10,7.265311e-01,5.763896e+10,6.542913e-01
6,0.50,10,"[520414.8342450353, -40545159636.12781, -60243...",1.456064e+23,-1.264635e+12,1.626068e+23,-9.752880e+11
7,0.50,50,"[2.2615762850427088e+21, -3.770982340548436e+3...",1.259542e+73,-1.093949e+62,1.406601e+73,-8.436553e+61
8,0.50,100,"[3.6686414568535576e+52, -6.124908928535069e+6...",3.322792e+135,-2.885942e+124,3.710745e+135,-2.225642e+124


C 5.1: At step size 0.01 the performance improves as iter goes from 10 to 50 to 100, showing slow and increasing progression in r2 score from negative value after 10 iter, to 0.62 at 100 iter.

At step size 0.10 after 10 iter the r2 value is 0.63, after 50 the r2 vlue is 0.65 and converges to near this value at step size 100 aswell.

At step size 0.50, the r2 value never positive, signaling the model prediction error is larger than the baseline error. This is because the step size is too large, the model acuratly converge.

At a high level, you dont want step size to be too small, because it takes many itnerations too find the min, and this uses more compute. You dont want step size to be large because then convergence cannot be found. In this instance step ize 0.1 was able to get to convergece the qucikest, only needing between 10-50 iter.

In [84]:
# C 6.2
def ridge_gradient_descent(X, y, alpha, iters, lam, penalize_intercept=False):
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1, 1)
    N, d = X.shape

    theta = np.zeros((d, 1), dtype=float)

    for _ in range(iters):

        grad = (2.0 / N) * (X.T @ (X @ theta - y))


        reg = 2.0 * lam * theta
        if not penalize_intercept:
            reg[0, 0] = 0.0

        theta = theta - alpha * (grad + reg)


        if not np.isfinite(theta).all():
            break

    return theta


In [85]:
#C 6.3
def closed_form_linear(X, y):
    return np.linalg.pinv(X) @ y

def closed_form_ridge(X, y, lam, penalize_intercept=False):
    d = X.shape[1]
    I = np.eye(d)
    if not penalize_intercept:
        I[0, 0] = 0.0
    A = X.T @ X + lam * I
    b = X.T @ y
    return np.linalg.solve(A, b)


np.random.seed(0)
N = 1000

x = np.random.uniform(-2, 2, size=N)


e = np.random.normal(0, np.sqrt(2), size=N)

y = 1 + 2*x + e

Xmat = np.c_[np.ones(N), x]
yvec = y.reshape(-1, 1)


rows = []


theta_ols = closed_form_linear(Xmat, yvec)
yhat_ols = (Xmat @ theta_ols).ravel()
rows.append(["OLS", 0, float(theta_ols[1]), mean_squared_error(y, yhat_ols), r2_score(y, yhat_ols)])

for lam in [1, 10, 100, 1000, 10000]:
    theta_r = closed_form_ridge(Xmat, yvec, lam, penalize_intercept=False)
    yhat = (Xmat @ theta_r).ravel()
    rows.append(["Ridge", lam, float(theta_r[1]), mean_squared_error(y, yhat), r2_score(y, yhat)])

results = pd.DataFrame(rows, columns=["Model", "lambda", "slope(theta1)", "MSE", "R2"])
results


/var/folders/tj/11696qnd2qv0jr8vrz4tm_t40000gn/T/ipykernel_86155/4090513944.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rows.append(["OLS", 0, float(theta_ols[1]), mean_squared_error(y, yhat_ols), r2_score(y, yhat_ols)])
/var/folders/tj/11696qnd2qv0jr8vrz4tm_t40000gn/T/ipykernel_86155/4090513944.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rows.append(["Ridge", lam, float(theta_r[1]), mean_squared_error(y, yhat), r2_score(y, yhat)])
/var/folders/tj/11696qnd2qv0jr8vrz4tm_t40000gn/T/ipykernel_86155/4090513944.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future.

,Model,lambda,slope(theta1),MSE,R2
0,OLS,0,1.965692,1.865374,0.736759
1,Ridge,1,1.964238,1.865377,0.736759
2,Ridge,10,1.951251,1.865656,0.736720
3,Ridge,100,1.830236,1.890166,0.733261
4,Ridge,1000,1.129641,2.809812,0.603481
5,Ridge,10000,0.233982,5.917267,0.164958


C6.3 As lambda got larger, the MSE increased, and the r2 value decreased